In [155]:
import pandas as pd
import numpy as np
import os

from sklearn.preprocessing import LabelEncoder, StandardScaler
from pycaret.regression import *
from gui import WyszukiwarkaNieruchomosciGUI
import tkinter as tk

import folium #Umożliwia tworzenie interaktywnych map w języku Python
from IPython.display import display # Umożliwia wyświetlanie interaktywnych wyników w środowisku IPython
from geopy.geocoders import Nominatim #służy do geokodowania, czyli przekształcania adresów lub miejsc na współrzędne geograficzne.
import requests
import polyline # używana do reprezentacji tras na mapie.
import pandas as pd
from folium.plugins import MarkerCluster, FeatureGroupSubGroup

# PDF
import jinja2
import pdfkit
from datetime import datetime
import pandas as pd
import time 

# Mail
import schedule
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

from selenium import webdriver
from selenium.webdriver.common.by import By

In [156]:
def rysowanie_mapki(df):
    '''Funkcja przyjmująca df, ktory musi zawierac kolumne ulice oraz Score. IM jako punkt startowy jest dopisywany automatycznie.
    Df w srodku sortuje sie wzgledem punktacji i ustawia kolorki ikonki itd odpowiednie dla oferty. 

    Zwraca liste w której pierwszy element to lista tekstów dotyczacych długosci podrozy, a drugi element to
    dfmapke z ofertami, ktore zostaly uzyte. Mapka wyświetla się automatycznie i tworzy plik html w miejscu gdzie jest plik z kodem, na mapie mamy
    sciezke miedzy glownymi ofertami + dolozone sa oferty ktore sa 'po drodze'. '''
    
    df=pd.concat([df,pd.DataFrame(data=[['plac Grunwaldzki 2, Wrocław',999]],columns=['ulica','Score'])],ignore_index=True)  #dodanie im jako startu
    kolorki = ['blue','green','green','green','orange','orange','orange','orange']
    ikonki = ['search','home','home','home','map-marker','map-marker','map-marker','map-marker']
    df=df.sort_values(by='Score',ascending=False)
    df=df[0:8]
    df['kolor']=kolorki
    df['ikona']=ikonki
    wykorzystane=df[1:4]
    mapbox_api_key = 'pk.eyJ1Ijoid3ljZW55IiwiYSI6ImNscnRraDBsajA3NXQyam5hdjloZTBoZ3EifQ.0gU_ozTPH70_eXfyfCAqkA'

    geolocator = Nominatim(user_agent="my_geocoder") #definiujemy usługę geokodowania
    
    loc = geolocator.geocode('Wrocław')
    loc = [loc.latitude,loc.longitude]
    
    map_wroclaw = folium.Map(loc, zoom_start=12, control_scale=True)
    # Dodawanie warstwy mapbox do mapy
    folium.TileLayer(
        tiles=f'https://api.mapbox.com/styles/v1/mapbox/streets-v11/tiles/256/{{z}}/{{x}}/{{y}}@2x?access_token={mapbox_api_key}',
        attr='Mapbox',
        name='Mapbox Streets',
        overlay=True
    ).add_to(map_wroclaw)
    

    coordinates=[]
    adres = []
    	
    marker_cluster = MarkerCluster().add_to(map_wroclaw)
    
    for i in range(0,8): #stawianie 3 glownych punktow i IM, i zbieranie koordynatow wszystkich
        adres.append(df.iloc[i]['ulica'])
        kolor = df.iloc[i]['kolor']
        ikona = df.iloc[i]['ikona']
        location = geolocator.geocode(adres[i])
        if i <= 3: #zeby na razie dodalo tylko im i 3 glowne punkty
            folium.Marker(location=[location.latitude, location.longitude], popup=adres[i],icon=folium.Icon(color=kolor, icon=ikona)).add_to(marker_cluster)
        coordinates.append([location.longitude,location.latitude])
    
    
    #szukanie optymalnej drogi miedzy im a trzema najlepszymi
    profile = 'driving'
    optimized_trips_url = f'https://api.mapbox.com/optimized-trips/v1/mapbox/{profile}/{coordinates[0][0]},{coordinates[0][1]};{coordinates[1][0]},{coordinates[1][1]};{coordinates[2][0]},{coordinates[2][1]};{coordinates[3][0]},{coordinates[3][1]}?access_token={mapbox_api_key}'
    response = requests.get(optimized_trips_url)
    data_opt = response.json()
    dystans = round(data_opt['trips'][0]['distance']/1000,2)
    czas = round(data_opt['trips'][0]['duration']/60)
    #ponizej sprawdzamy dla pozostalych 4 opcjonlanych miejsc czy dolozenie ich 
    #zwiekszy czas przejazdu o poonad 20% - jesli tak olewamy, jesli nie dokladamy.
    czas_dodatkowy=[]
    for i in range(4,8):
        profile = 'driving'
        optimized_trips_url = f'https://api.mapbox.com/optimized-trips/v1/mapbox/{profile}/{coordinates[0][0]},{coordinates[0][1]};{coordinates[1][0]},{coordinates[1][1]};{coordinates[2][0]},{coordinates[2][1]};{coordinates[3][0]},{coordinates[3][1]};{coordinates[i][0]},{coordinates[i][1]}?access_token={mapbox_api_key}'
        response = requests.get(optimized_trips_url)
        data_opt = response.json()
        if data_opt['trips'][0]['duration']/60 <= czas*1.2:
            czas_dodatkowy.append([data_opt['trips'][0]['duration']/60 - czas,i])
    #sprawdzanie ile znalazlo i tworzenie z nimi nowej drogi
    #wpisywanie wspolrzednych do stringa i wrzucanie znacznikow dodatkowych miejsc na mape
    pozostale = '' 
    for i in range (0,len(czas_dodatkowy)):
        wykorzystane=pd.concat([wykorzystane,df.iloc[czas_dodatkowy[1][1]].to_frame().T],ignore_index=True) 
        pozostale = pozostale + ';' +str(coordinates[czas_dodatkowy[i][1]][0])+','+str(coordinates[czas_dodatkowy[i][1]][1])
        folium.Marker(location=[coordinates[czas_dodatkowy[i][1]][1], coordinates[czas_dodatkowy[i][1]][0]],popup=adres[czas_dodatkowy[i][1]],icon=folium.Icon(color='orange', icon='map-marker')).add_to(marker_cluster)
    
    #oficjalna trasa z punktami ktore sie oplaca
    optimized_trips_url = f'https://api.mapbox.com/optimized-trips/v1/mapbox/{profile}/{coordinates[0][0]},{coordinates[0][1]};{coordinates[1][0]},{coordinates[1][1]};{coordinates[2][0]},{coordinates[2][1]};{coordinates[3][0]},{coordinates[3][1]}{pozostale}?access_token={mapbox_api_key}'
    
    #reszta rzeczy zeby mapa sie pojawila i stringi z informacja ile sie jedzie bez / z punktami dodatkowymi.
    response = requests.get(optimized_trips_url)
    data_opt = response.json()
    coor = polyline.decode(data_opt['trips'][0]['geometry'])
    folium.PolyLine(locations=coor, color='blue').add_to(map_wroclaw)
    teksty = [
    f"Odległość trasy bez dodatkowych ofert: ok. {dystans} km",
    f"Czas podróży bez dodatkowych ofert: ok. {czas} min",
    f"Odległość trasy: ok. {round(data_opt['trips'][0]['distance']/1000,2)} km",
    f"Czas podróży: ok. {round(data_opt['trips'][0]['duration']/60)} min"
    ]
    
    
    map_wroclaw.save('map.html')
    display(map_wroclaw)
    return [teksty,wykorzystane]

In [157]:
data = pd.read_csv('./combined_24.csv', index_col=0)
data_col = data.columns
columns_to_check_duplicates = data.columns.difference(['link'])
data0 = data[columns_to_check_duplicates].drop_duplicates()
data0 = data0.join(data['link'])[data_col]


In [158]:
many_nans = (data0.isna().sum(axis=1)/data0.shape[1]).sort_values(ascending = False)
many_nans_ids = many_nans[many_nans > 0.7].index
data0.drop(many_nans_ids, inplace=True)
l = len(data0[data0.cena.isna()])
data0['cena'].fillna(0, inplace=True)


In [159]:
root = tk.Tk()
app = WyszukiwarkaNieruchomosciGUI(root)
root.mainloop()

wanted_lok = app.saved_lokalizacja

try:
    wanted_price = int(app.saved_cena_max)
except ValueError:
    wanted_price = 1e8

try:
    wanted_pow_min = int(app.saved_powierzchnia_od)
except ValueError:
    wanted_pow_min = 0

try:
    wanted_pow_max = int(app.saved_powierzchnia_do)
except ValueError:
    wanted_pow_max = 1000

try:
    wanted_rooms = int(app.saved_pokoje)
except ValueError:
    wanted_rooms = 1
pow = [0.8 * wanted_pow_min, 1.2 * wanted_pow_max]



In [160]:
# print(wanted_lok)
# print(wanted_price)
# print(wanted_pow_min)
# print(wanted_pow_max)
# print(wanted_rooms)

In [161]:
data_raw = data0.copy()
colnames = ['cena','powierzchnia']
replace_dict = {'zł': '', 'm2': '', '/': '', ' ': ''}

data_raw[colnames] = data_raw[colnames].astype('str')
data_raw[colnames] = data_raw[colnames].applymap(lambda x: str(x).replace('zł', '').replace('m²', '').replace('/', '').replace(' ', '').replace(',', '.'))
data_raw[colnames] = data_raw[colnames].astype('float')
data_raw['piętro'] = data_raw['piętro'].str.replace('parter', '0')
data_raw['liczba pięter'] = data_raw['piętro'].str.extract(r'(\d+)/(\d+)')[1].astype(float)
data_raw['piętro'] = data_raw['piętro'].str.extract(r'(\d+)').astype(float)
data_raw['pokoje'] = data_raw['pokoje'].fillna(1)
data_raw['rok budowy'] = 1969
data_raw = data_raw[((data_raw.cena < 1.1 * wanted_price) | data_raw.cena.isna()) & (data_raw.pokoje >= 2) & (data_raw.powierzchnia.between(pow[0], pow[1]))]
#data_raw.isna().sum()

In [162]:
dzielnice = '|'.join(['Fabryczna', 'Krzyki', 'Stare Miasto', 'Śródmieście', 'Psie Pole'])
data_raw['dzielnica'] = data_raw['lokalizacja'].str.extract(f'({dzielnice})')
data_raw['dzielnica'].fillna('Stare Miasto', inplace = True)
data_raw.insert(4, 'dzielnica', data_raw.pop('dzielnica'))
data_raw['rok budowy'] = 1969

data_raw['piętro'] = data_raw['piętro'].fillna(4)
data_raw['liczba pięter'] = data_raw[['liczba pięter', 'piętro']].max(axis=1).fillna(4)

column_moved = data_raw.columns[-1]
data_raw.insert(12, column_moved, data_raw.pop(column_moved))

data_raw['ulica'] = np.where(data_raw['lokalizacja'].str.contains('ul.'),
                             data_raw['lokalizacja'].str.extract(r'ul\.(.*?),', expand=False),
                             np.nan)

# Run the second line only for rows where 'ulica' is NaN
mask = data_raw['ulica'].isna()
data_raw.loc[mask, 'ulica'] = np.where(
    data_raw.loc[mask, 'nazwa ogloszenia'].str.contains('ul.'),
    data_raw.loc[mask, 'nazwa ogloszenia'].str.extract(r'ul\.(.*?),', expand=False),
    np.nan
)
data_raw.ulica = data_raw.ulica.str.strip()
data_raw['stan wykonczenia'].fillna('do remontu', inplace = True)
data_raw[['ulica', 'lokalizacja', 'nazwa ogloszenia']]
dzielnica_dict = {'Psie Pole': 'Zielna', 'Stare Miasto': 'Zamkowa', 'Fabryczna': 'Rudzka', 'Krzyki': 'Gajowa', 'Śródmieście': 'Kossaka'}
data_raw['ulica_dzielnica'] = data_raw['dzielnica'].map(dzielnica_dict)
data_raw['ulica'] = data_raw['ulica'].fillna(data_raw['ulica_dzielnica'])
data_raw['balkon/ogrod/taras'] = ~data_raw['balkon/ogrod/taras'].isna()
data_raw['miejsce parkingowe'] = ~data_raw['miejsce parkingowe'].isna()
wyk_dict = {'do remontu': 'Do remontu', 'do wykończenia': 'Do odświeżenia', 'do zamieszkania': 'Gotowe do wprowadzenia'}
data_raw['stan wykonczenia'] = data_raw['stan wykonczenia'].map(wyk_dict)
data_raw['stan wykonczenia'].fillna('Do remontu', inplace = True)

median_dict = data_raw.groupby('pokoje')['powierzchnia'].median().to_dict()
data_raw['powierzchnia'] = data_raw.apply(
    lambda row: row['powierzchnia'] if pd.notna(row['powierzchnia']) else median_dict.get(row['pokoje']),
    axis=1
)
data_raw0 = data_raw[['cena', 'ulica', 'pokoje', 'powierzchnia', 'piętro', 'liczba pięter', 'stan wykonczenia', 'balkon/ogrod/taras', 'miejsce parkingowe', 'rok budowy']]
data_raw0['ulica'].fillna('pl. Dominikański', inplace = True)
# print(data_raw.shape)
# data_raw.isna().sum()

In [163]:
model_data = data_raw0.copy()
model_data['stan wykonczenia'] = model_data['stan wykonczenia'].map({'Do remontu': 0, 'Do odświeżenia': 1, 'Gotowe do wprowadzenia': 2})
label_encoder = LabelEncoder()
model_data['ulica'] = label_encoder.fit_transform(model_data['ulica'])

In [164]:
loaded_model = load_model('model0')
try:
    predicted = predict_model(loaded_model, data=model_data[model_data.cena == 0])
    predicted.drop(columns=['cena'], inplace=True)
    pred_ids = predicted.index
    predicted.rename(columns={'prediction_label': 'cena'}, inplace=True)
    data_model = pd.concat([model_data[model_data.cena != 0], predicted])
    print('Predicted')
except ValueError:
    data_model = model_data.copy()
data_model = data_model[((data_model.cena < 1.2 * wanted_price) | data_model.cena.isna())]
#data_model 

Transformation Pipeline and Model Successfully Loaded
Predicted


In [165]:
loaded_model2 = load_model('model1')
new_prices = predict_model(loaded_model2, data = data_model)
new_prices['ulica'] = label_encoder.inverse_transform(new_prices['ulica'])
new_prices['stan wykonczenia'] = new_prices['stan wykonczenia'].map({0: 'Do remontu', 1: 'Do odświeżenia', 2: 'Gotowe do wprowadzenia'})
ids = new_prices.index
#new_prices

Transformation Pipeline and Model Successfully Loaded


In [166]:
to_rate = data_raw[['dzielnica', 'pokoje', 'powierzchnia', 'stan wykonczenia', 'balkon/ogrod/taras', 'miejsce parkingowe', 'cena']]
to_rate = to_rate.join(new_prices['prediction_label'])#.loc[ids]
#to_rate

In [167]:
to_rate = data_raw[['dzielnica', 'pokoje', 'powierzchnia', 'stan wykonczenia', 'balkon/ogrod/taras', 'miejsce parkingowe', 'cena']]
to_rate = to_rate.join(new_prices['prediction_label'])

if wanted_lok != '':
    to_rate['wanted_loc'] = to_rate['dzielnica'] == wanted_lok
else:
    to_rate['wanted_loc'] = True

to_rate['pokoje_bon'] = to_rate['pokoje'] > wanted_rooms
to_rate['stan wykonczenia'] = to_rate['stan wykonczenia'].map({'Do remontu': 0, 'Do odświeżenia': 1, 'Gotowe do wprowadzenia': 2})
#to_rate

In [168]:
weights = {
    'pokoje': 2,
    'powierzchnia': 2,
    'stan wykonczenia': 2,
    'balkon/ogrod/taras': 1,
    'miejsce parkingowe': 1,
    'cena': -4,  
    'prediction_label': -2,
    'wanted_loc': 3,
    'pokoje_bon': 2
}

numeric_columns = to_rate.select_dtypes(include=['number']).columns
normalized_data = to_rate.copy()
for column in numeric_columns:
    if (to_rate[column].max() - to_rate[column].min()) != 0:
        normalized_data[column] = (to_rate[column] - to_rate[column].min()) / (to_rate[column].max() - to_rate[column].min())
    else:
        normalized_data[column] = 1

normalized_data['Ocena'] = sum(normalized_data[column] * weights[column] for column in weights)
score_min = normalized_data['Ocena'].min()
score_max = normalized_data['Ocena'].max()
normalized_data['Ocena'] = round(((normalized_data['Ocena'] - score_min) * (10 / (score_max - score_min))),2)
best = normalized_data[['dzielnica', 'Ocena']].sort_values('Ocena', ascending=False)[:7]
#best

In [169]:
used_cols = ['nazwa ogloszenia', 'cena', 'lokalizacja', 'dzielnica', 'ulica', 'pokoje', 'powierzchnia', 'rodzaj zabudowy','telefon', 'link', 'Ocena']
final = data.join(best, how = 'right')
final['ulica'] = np.where(final['lokalizacja'].str.contains('ul.'),
                             final['lokalizacja'].str.extract(r'ul\.(.*?),', expand=False),
                             np.nan)

# Run the second line only for rows where 'ulica' is NaN
mask = final['ulica'].isna()
final.loc[mask, 'ulica'] = np.where(
    final.loc[mask, 'nazwa ogloszenia'].str.contains('ul.'),
    final.loc[mask, 'nazwa ogloszenia'].str.extract(r'ul\.(.*?),', expand=False),
    np.nan
)
final.ulica = final.ulica.str.strip()
dzielnice = '|'.join(['Fabryczna', 'Krzyki', 'Stare Miasto', 'Śródmieście', 'Psie Pole'])
final['dzielnica'] = final['lokalizacja'].str.extract(f'({dzielnice})')
final.insert(4, 'dzielnica', final.pop('dzielnica'))
final.insert(5, 'ulica', final.pop('ulica'))
dzielnica_dict = {'Psie Pole': 'Zielna', 'Stare Miasto': 'Zamkowa', 'Fabryczna': 'Rudzka', 'Krzyki': 'Gajowa', 'Śródmieście': 'Kossaka'}
final['ulica_dzielnica'] = final['dzielnica'].map(dzielnica_dict)
final['ulica'] = final['ulica'].fillna(final['ulica_dzielnica'])
final['ulica'] = final['ulica'] + ', Wrocław'


In [170]:
#final[used_cols]
#final.to_csv('sample4map3.csv')

In [171]:
idd = final[final.cena.isna()].index
final['cena'].fillna(round(predicted.loc[idd].cena, 0).astype(int), inplace = True)
final['cena'] = final['cena'].astype(str).str.strip('.0')
final.loc[~final['cena'].str.contains(' zł'), 'cena'] = final['cena'] + ' zł*'
#final.loc[final['cena'].str.contains('\*'), 'cena'] = final['cena'][:3] + ' ' + final['cena'][3:]


In [172]:
df = final[used_cols]
print(df)

                                      nazwa ogloszenia          cena  \
7             5 pokoi z balkonem/tereny zielone/gotowe  1 022 925 zł   
87                                  Opis nieruchomości   1014657 zł*   
15   3-pokojowe mieszkanie w centrum Wrocławia- bezPcc    949 000 zł   
10    Przestronne 3-pok. mieszkanie 75,2 m2, 2 balkony    940 000 zł   
32               Widok na zieleń | Ostatnie mieszkania    724 259 zł   
68                                  Opis nieruchomości   1079364 zł*   
118                                 Opis nieruchomości   1047177 zł*   

                                           lokalizacja    dzielnica  \
7    ul. Jordanowska, Oporów, Fabryczna, Wrocław, d...    Fabryczna   
87                                             Krzyki,       Krzyki   
15   ul. Bolesława Prusa, Nadodrze, Śródmieście, Wr...  Śródmieście   
10   ul. Jurija Gagarina, Muchobór Wielki, Fabryczn...    Fabryczna   
32   ul. Maślicka, Maślice, Fabryczna, Wrocław, dol...    Fabryczna 

In [173]:
rys=rysowanie_mapki(df)

In [174]:
def generate_pdf(df, driver_path, wkhtmltopdf_path):
    """Generuje plik PDF na podstawie podanej ramki danych dotyczącej nieruchomości."""

    cdir = os.getcwd()


    # zbieranie linków do zdjęć
    img_urls = []

    # print('shape', df.shape[0])
    # print('linki', df['link'])

    drop = [] # wiersze ofert w df, które są nieaktualne

    for url in df['link']:

        driver = webdriver.Chrome()
        driver.get(url)

        try:
            img = driver.find_element(By.CLASS_NAME, 'image-gallery-image')
            img_urls.append(img.get_attribute("src"))
        except:
            drop.append(url)

        driver.quit()

    df_new = df[~df.isin(drop).any(axis=1)]   # usuwanie linków, które nie przeszły
    # print(df)

    html_mail = ''''''

    with open(cdir + '/html_mail.html', 'w', encoding='utf-8') as file:
        file.write(html_mail)

    # Modyfikacja pliku html
        
    # print('shape', df.shape[0])
    # print('linki' , len(df['link']))
    # print('linki' , len(img_urls))

    with open(cdir + '/html_mail.html', 'r') as file:
        html_content = file.read()

        for i in range(df_new.shape[0]):
            
            # zbieranie zdjęć
            # jeśli oferta nieaktualna to pomijamy ofertę

            img_path = img_urls[i]
                        
            # print(img_urls[i])

            # uzpełnianie opisu
            lokalizacja = df_new.iloc[i]['lokalizacja']
            cena = df_new.iloc[i]['cena']
            powierzchnia = df_new.iloc[i]['powierzchnia']
            pokoje = df_new.iloc[i]['pokoje']
            r_zabudowy = df_new.iloc[i]['rodzaj zabudowy']
            telefon = df_new.iloc[i]['telefon']
            link_o = df_new.iloc[i]['link']

            html_code = \
                '''

                <h2 style="text-align: center;">Oferta nr {}</h2>
                
                <img align="center" src="{}" alt="Oferta" width="600" height="400" class="ImageCenter" class="ImageBorder">

                <p style="padding-left: 10px;">              

                <br /><strong>Lokalizacja</strong>: {}
                <br /><strong>Cena</strong>: {}
                <br /><strong>Powierzchnia</strong>: {}
                <br /><strong>Liczba pokoi</strong>: {}
                <br /><strong>Rodzaj zabudowy</strong>: {}
                <br /><strong>Telefon</strong>: {}
                <br /><strong>Link</strong>: <a href="{}">{}</a></p>
                
                <p style="padding-left: 10px;">&nbsp;</p>
                <p>&nbsp;</p>
                '''
                
            formatted_html_code = html_code.format(i+1, img_path, lokalizacja, cena, powierzchnia, pokoje, r_zabudowy, telefon, link_o, link_o)

            html_content += formatted_html_code

    html_preamble = '''
    <html>

    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link rel="stylesheet" href="https://www.w3schools.com/w3css/4/w3.css">
    <link rel="stylesheet" href="https://www.w3schools.com/lib/w3-theme-teal.css">

    <head>
        <meta charset="utf-8">
    </head>

    <body>

    <div class="w3-card-4">

    <div class="w3-container w3-theme w3-card">
    <h1 style="text-align: center;">Raport nieruchomości {}</h1>
    </div>

    <div class="w3-theme-l5">

    </div>
    '''

    data = datetime.now().strftime('%d-%m-%Y')
    html_preamble = html_preamble.format(data)

    html_tail = '''

    <br />
    <br />

    </div>

    </body>
    </html>
    '''

    with open(cdir + '/html_mail.html', 'w', encoding='utf-8') as file:
        file.write(html_preamble)
        file.write(html_content)
        file.write(html_tail)

        # Genereowanie PDF

    template_loader = jinja2.FileSystemLoader(cdir)
    template_env = jinja2.Environment(loader=template_loader)
    template = template_env.get_template('html_mail.html')

    pdf_content = template.render()

    config = pdfkit.configuration(wkhtmltopdf=wkhtmltopdf_path)
    output_pdf_path = cdir + '/Raport.pdf'  
    pdfkit.from_string(pdf_content, output_pdf_path, configuration=config,
                        css=cdir + '/style.css', options={"enable-local-file-access": ""})

In [177]:
# Wysyłanie Maila

def sent_mail(sender, receiver, body, password_path):
    """Wysyła maila z ofertami spełniającymi kryteria użytkownika. Sender - mail nadawcy, receiver - mail odbiorcy, body - treść maila.
    Ścieżka password_path prowadzi do pliku txt z hasłem dostępu do konta Google."""

    pdf_dir = os.getcwd() + '/Raport.pdf'

    msg = MIMEMultipart()              # pozwala na dodawanie plików do maila
    msg['From'] = sender
    msg['To'] = receiver
    msg['Subject'] = 'Python Projekt'

    with open(password_path, "r") as f:
        password = f.readline()
    
    # Treść maila
    msg.attach(MIMEText(body, 'plain'))

    # Dodanie załącznika PDF
    attachment = open(pdf_dir, 'rb')
    part = MIMEBase('application', 'octet-stream')
    part.set_payload((attachment).read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', f'attachment; filename = Raport.pdf')
    msg.attach(part)
    
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        smtp.login(sender, password)
        smtp.sendmail(sender, receiver, msg.as_string())  # msg.as_string() - zapewnia poprawny format maila

In [178]:
# generate_pdf(df, 'C:/Program Files (x86)/chromedriver.exe', 'C:/Program Files/wkhtmltopdf/bin/wkhtmltopdf.exe')
sent_mail('Raport.wyceny@gmail.com', 'piotr.granieczny26@gmail.com', 'Twój raport nieruchomości.', 'D:/Jupyter/MM Python - projekt/mail/password.txt')